In [1]:
from sentence_transformers import SentenceTransformer, util
import os
os.chdir('../')
os.environ["TRANSFORMERS_CACHE"]="/common/users/zl502/huggingface/cache/"
from general_dataset import GeneralDataset

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoFeatureExtractor

from torch.utils.data import DataLoader
import torch
from agi_utils import *
import torch

/research/cbim/vast/zl502/anaconda3/envs/peft_agi/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /research/cbim/vast/zl502/anaconda3/envs/peft_agi/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
data_path = "/common/users/zl502/openagi_data/"
task_discriptions = txt_loader("./task_description.txt")
# task_idx = [0,21,61,105,110,120,10,35,62,107,115]
test_task_idx = [2,3,10,15,20,35,45,55,65,70,70,90,106,107]
test_dataloaders = []
for i in test_task_idx:
    dataset = GeneralDataset(i, data_path)
    dataloader = DataLoader(dataset, batch_size=20)
    test_dataloaders.append(dataloader)
    
test_tasks = [task_discriptions[i].strip() for i in test_task_idx]

In [3]:
train_solution = []
with open('./train_model_sequence.txt') as f:
    lines = f.readlines()
    for line in lines[:50]:
        train_solution.append(line)
f.close()

train_tasks = []
with open('./train_task_description.txt') as f:
    lines = f.readlines()
    for line in lines[:50]:
        train_tasks.append(line)
f.close()

context = ""
for i in range(len(train_tasks)):
    if i >= 15:
        break
    steps = ""
    for index,j in enumerate(train_solution[i].split(',')):
        steps += "Step " + str(index+1) + ":" + j.strip("\n") + ", \n"
    cur = "Problem: " + train_tasks[i] +  "Solution:\n" + steps
    context += cur
    
# print(context + "###Human: " + test_tasks[0]+"\###Assistant: ")

In [4]:
import transformers

base_model = "meta-llama/Llama-2-13b-chat-hf"

cache_dir = "/common/users/zl502/huggingface/cache"
# base_model = "TheBloke/Llama-2-13B-chat-GGML"
# base_model = "chainyo/alpaca-lora-7b"
load_8bit = True

hf_token = "YOUR TOKEN"

# max_memory_mapping = {0: "18GB", 1: "18GB", 2: "18GB", 3: "18GB", 4: "18GB", 5: "18GB", 6: "18GB", 7: "18GB",}
max_memory_mapping = {
    0: "48GB", 
    1: "48GB", 
    2: "48GB", 
    3: "48GB", 
    4: "0GB", 
    5: "0GB", 
    6: "0GB", 
    7: "0GB",
}

tokenizer = AutoTokenizer.from_pretrained(
    base_model, 
    cache_dir=cache_dir, 
    use_auth_token=hf_token
)

tokenizer.pad_token_id = 0

model = AutoModelForCausalLM.from_pretrained(
    base_model, 
    cache_dir=cache_dir,
    # load_in_8bit=True,\
    device_map='auto',
    max_memory = max_memory_mapping,
    use_auth_token=hf_token
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# tokenizer= torch.nn.DataParallel(tokenizer,device_ids = [0,1,2,3])
# model= torch.nn.DataParallel(model,device_ids = [0,1,2,3])

In [6]:
import numpy as np
from IPython.utils import io
from agi_utils import *
import random
from tqdm import tqdm
from evaluate import load
from torchvision import transforms
from transformers import AutoModel, AutoFeatureExtractor
from torchmetrics.multimodal import CLIPScore
from combine_module_seq import SeqCombine


clip_score = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

# Load a pre-trained Vision Transformer model and its feature extractor
vit_ckpt = "nateraw/vit-base-beans"
vit = AutoModel.from_pretrained(vit_ckpt)
vit.eval()
vit_extractor = AutoFeatureExtractor.from_pretrained(vit_ckpt)

f = transforms.ToPILImage()
bertscore = load("bertscore")

# device_list = ["cuda:1","cuda:2","caugment_prompt:3","cuda:4","cuda:5","cuda:7","cpu"]
device_list = ["cuda:5", "cpu"]
seqCombination = SeqCombine(device_list)

Some weights of the model checkpoint at nateraw/vit-base-beans were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


In [7]:
import openai
# prompt_length = len(input_text)
openai.api_key = "YOUR TOKEN"

def generate_module_list_with_gpt(generated_module_seq):
    todo_prompt = "You are a key phrase extractor who is able to extract potential module names from the given context. You have already known all the module names in the full module list. The full module list is: [Image Classification, Colorization, Object Detection, Image Deblurring, Image Denoising, Image Super Resolution, Image Captioning, Text to Image Generation, Visual Question Answering, Sentiment Analysis, Question Answering, Text Summarization, Machine Translation]. Given the following context: '{}'. Please extract a module sequence from this context and remove module names which do not exist in the full module list from this sequence. Output the module sequence after filtering as the format of 'module: module1, module: module2, module: module3, etc...'. "
    prompt = todo_prompt.format(generated_module_seq)

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    content = completion.choices[0].message["content"]
    
    # print(content)
    
    content = content.split("module: ")[1:]
    
    result = ""
    for c in content:
        result += c
    
    # result = result[:-1] if len(result) > 0 else result
    
    return result

In [8]:
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")


rewards = []
clips = []
berts = []
similarities = []

module_length = 10
num_beams = 1
num_return_sequences = 1


eval_device = torch.device("cuda:4")

for i, task_description in enumerate(tqdm(test_tasks)):
    # if i == 1:
    #     break
    # print(task_description)
    task_rewards = []
    with torch.no_grad():
        input_s = [context + "Problem: " + task_description + "Solution:\n"]
        # print(input_s)
        input_ids = tokenizer.batch_encode_plus(
            input_s, padding="longest", return_tensors="pt"
        )["input_ids"].to(eval_device)
        output = model.generate(
            input_ids=input_ids,
            max_length=2048, return_dict_in_generate=True, output_scores=True, output_hidden_states=True,repetition_penalty=1.25
        )
    generated_seq = tokenizer.decode(
        output["sequences"][0], skip_special_tokens=True, temperature=0, top_p=0.8, repetition_penalty=1.25
    )

    # print(generated_seq)
    
    llama_solution = generated_seq.split("Problem: ")[1]
    # print(llama_solution)
    llama_solution = llama_solution.split("\n")

    llama_steps = []
    for l,j in enumerate(llama_solution):
        if j[0:4] == "Step":
            llama_steps.append(llama_solution[l])

    print(llama_steps)
    # llama_steps = generate_module_list_with_gpt(generated_seq[len(input_s[0]):]).split(",")
    module_list = match_module_seq(llama_steps, sentence_model)
    print("Module list: ", module_list)

    if len(module_list) >= 1 and whole_module_seq_filter(module_list, test_task_idx[i]):
        seqCombination.construct_module_seq(module_list)

        for idx, batch in tqdm(enumerate(test_dataloaders[i])):
            inputs = list(batch['input'][0])
            # print("Inputs: ", inputs)
            try:
                predictions = seqCombination.run_module_seq(inputs)
            except:
                ave_task_reward = 0
                break

            if 0 <= test_task_idx[i] <= 14:
                outputs = list(batch['output'][0])
                dist = image_similarity(predictions, outputs, vit, vit_extractor)
                task_rewards.append(dist / 100)
            elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
                outputs = list(batch['output'][0])
                f1 = np.mean(txt_eval(predictions, outputs, bertscore, device=eval_device))
                
                task_rewards.append(f1)
            else:
                score = clip_score(predictions, inputs)
                task_rewards.append(score.detach() / 100)
                
        ave_task_reward = np.mean(task_rewards)    
        seqCombination.close_module_seq()
            
    else:
        ave_task_reward = 0

    print(ave_task_reward)
        
    if 0 <= test_task_idx[i] <= 14:
        similarities.append(ave_task_reward)
    elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
        berts.append(ave_task_reward)
    else:
        clips.append(ave_task_reward)

    rewards.append(ave_task_reward)     
    

print("Finished testing!")        

  0%|                                                    | 0/14 [00:00<?, ?it/s]

['Step 1:Image Super Resolution, ', 'Step 2: Image Denoising, ', 'Step 3: Image Deblurring, ', 'Step 4: Colorization, ']
Module list:  Image Super Resolution, Image Denoising, Image Deblurring, Colorization



0it [00:00, ?it/s]
1it [00:46, 46.40s/it]
2it [01:28, 44.16s/it]
3it [02:11, 43.38s/it]
4it [02:55, 43.45s/it]
5it [03:38, 43.65s/it]
  7%|███                                        | 1/14 [04:17<55:43, 257.16s/it]

0.7065360565185547
['Step 1:Image Super Resolution, ', 'Step 2: Image Denoising, ', 'Step 3: Image Deblurring, ', 'Step 4: Colorization, ']
Module list:  Image Super Resolution, Image Denoising, Image Deblurring, Colorization



0it [00:00, ?it/s]
1it [00:42, 42.26s/it]
2it [01:23, 41.78s/it]
3it [02:06, 42.09s/it]
4it [02:48, 42.26s/it]
5it [03:32, 42.53s/it]
 14%|██████▏                                    | 2/14 [08:51<53:26, 267.20s/it]

...

Module list:  Image Super Resolution, Image Denoising, Image Deblurring, Colorization
0
['Step 1:Image Super Resolution, ', 'Step 2: Image Denoising, ', 'Step 3: Image Deblurring, ', 'Step 4: Colorization, ']


 93%|███████████████████████████████████████▉   | 13/14 [25:41<01:11, 71.88s/it]

Module list:  Image Super Resolution, Image Denoising, Image Deblurring, Colorization
0
['Step 1:Image Super Resolution, ', 'Step 2: Image Denoising, ', 'Step 3: Image Deblurring, ', 'Step 4: Colorization, ']


100%|██████████████████████████████████████████| 14/14 [25:59<00:00, 111.40s/it]

Module list:  Image Super Resolution, Image Denoising, Image Deblurring, Colorization
0
Finished testing!


In [9]:
# print(input_s[0])
# print(generated_seq[len(input_s[0]):])
# vicuna_steps = generate_module_list_with_gpt(generated_seq[len(input_s[0]):]).split(",")
# module_list = match_module_seq(vicuna_steps, sentence_model)
# print(module_list)

In [10]:
avg_clips = np.mean(clips)
avg_berts = np.mean(berts)
avg_similarities = np.mean(similarities)
avg_rewards = (avg_clips + avg_berts + avg_similarities) / 3
print([avg_clips, avg_berts, avg_similarities, avg_rewards])

[0.0, 0.0, 0.7024120686848958, 0.2341373562282986]
